In [11]:
from food.milvus import collection

from time import sleep
from food.tools import from_pickle
from food.psql import *
import pandas as pd
from tqdm import tqdm
import custom_pandas as cpd


In [13]:
engine.execute('truncate table indexed')

In [14]:

query = f"""select f.id, f.clip
        FROM      foods             f
        LEFT JOIN indexed i on (f.id = i.id)
        WHERE f .clip    is not null and i.indexed is null
        """
limit = 100000
for df in tqdm(cpd.read_sql_query(query, engine, chunksize=limit), desc="milvus_update"):
    df = df[['id','clip']]
    collection.insert(df)
    to_insert = df[['id']]
    to_insert['indexed'] = True
    to_insert.to_sql("indexed",engine,index=False,if_exists = 'append',method = 'multi')


collection.num_entities

milvus_update: 1it [00:02,  2.68s/it]


7083

In [20]:
collection.has_index()

False